In [77]:
import cv2
from PIL import Image, ImageDraw
import pandas as pd

# Load the Excel file into a dataframe
dataframe = pd.read_excel("C:/Users/Lenovo/Desktop/Unic_Kharkiv/Python/Lab6/lab6.xlsx")

# Extract data for Variant #4
variant_data = dataframe[dataframe["N"] == 4].iloc[0]
file_name = variant_data["file name"]
output_size = tuple(map(int, variant_data["image size"].split("x")))
glasses_color = variant_data["glasses color"]
line_width = int(variant_data["line width"])

# Load the image
image_path = "C:/Users/Lenovo/Desktop/Unic_Kharkiv/Python/Lab6/longbottom.jpg"
original_image = cv2.imread(image_path)
if original_image is None:
    print(f"Failed to load image from {image_path}")
    exit()

gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

# Haar cascade for face and eyes detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# Detect faces and eyes
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
if len(faces) > 0:
    (x, y, w, h) = faces[0]  # Only process the first detected face
    roi_gray = gray_image[y:y + h, x:x + w]
    eyes = eye_cascade.detectMultiScale(roi_gray)

    # Function to add round glasses
    def add_round_glasses(image, eyes_coordinates, glasses_color, line_width, output_size, output_file_path):
        # Convert OpenCV image to PIL
        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Sort eyes by x-coordinate to identify left and right eyes
        eyes_coordinates = sorted(eyes_coordinates, key=lambda e: e[0])

        # Draw glasses if exactly two eyes are detected
        if len(eyes_coordinates) == 2:
            (ex1, ey1, ew1, eh1) = eyes_coordinates[0]  # Left eye
            (ex2, ey2, ew2, eh2) = eyes_coordinates[1]  # Right eye

            # Calculate center and radius for each eye
            center1 = (x + ex1 + ew1 // 2, y + ey1 + eh1 // 2)
            radius1 = ew1 // 2

            center2 = (x + ex2 + ew2 // 2, y + ey2 + eh2 // 2)
            radius2 = ew2 // 2

            # Draw left and right glasses frames
            draw.ellipse(
                [(center1[0] - radius1, center1[1] - radius1), 
                 (center1[0] + radius1, center1[1] + radius1)],
                outline=glasses_color, width=line_width
            )
            draw.ellipse(
                [(center2[0] - radius2, center2[1] - radius2), 
                 (center2[0] + radius2, center2[1] + radius2)],
                outline=glasses_color, width=line_width
            )

            # Draw the bridge connecting the glasses
            draw.line(
                [(center1[0] + radius1, center1[1]), 
                 (center2[0] - radius2, center2[1])],
                fill=glasses_color, width=line_width
            )

            # Optional: Add arms extending outward
            arm_length = radius1
            draw.line(
                [(center1[0] - radius1, center1[1]), 
                 (center1[0] - radius1 - arm_length, center1[1])],
                fill=glasses_color, width=line_width
            )
            draw.line(
                [(center2[0] + radius2, center2[1]), 
                 (center2[0] + radius2 + arm_length, center2[1])],
                fill=glasses_color, width=line_width
            )

        # Center the face and resize
        cropped_image = pil_image.crop((x, y, x + w, y + h)).resize(output_size)
        cropped_image.save(output_file_path)
        print(f"Image saved with glasses at {output_file_path}")

    # Define output file path
    output_file_path = "C:/Users/Lenovo/Desktop/Unic_Kharkiv/Python/Lab6/longbottom_with_glasses.jpg"

    # Add glasses and save the image
    add_round_glasses(original_image, eyes, glasses_color, line_width, output_size, output_file_path)
else:
    print("No faces or eyes detected in the image.")


Image saved with glasses at C:/Users/Lenovo/Desktop/Unic_Kharkiv/Python/Lab6/longbottom_with_glasses.jpg
